# Astrophysics Literacy: NLP Pipeline



1.   Preprocessing
2.   Text Normalization
3.   NLP Piepleine Implementation 
4.   Keyword Extractions
5.   Word Embeddings (Advanced Keyword Extractions)
6.   ML Modelling



## Part 1: Preprocessing

In [ ]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install pytextrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_lg
!{sys.executable} -m spacy download en_core_web_md
!{sys.executable} pip install scispacy
!{sys.executable} -m spacy download en_core_sci_sm

2022-08-02 16:41:15.837055: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
2022-08-02 16:42:20.562715: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
/usr/bin/python3: can't open file 'pip': [Errno 2] No such file or directory
2022-08-02 16:42:34.712946: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] 

In [ ]:
import spacy
from collections import Counter
from string import punctuation
import en_core_web_lg
import en_core_web_sm
import en_core_web_md

In [ ]:
 #variate for different usages
 #we use en_core_web_lg for slightly better precision although it makes the process 79 times slower than en_core_web_sm.
 nlpLg = spacy.load("en_core_web_lg")
 nlpSm = spacy.load("en_core_web_sm")

In [ ]:
import json

In [ ]:
#80k data
with open('/content/drive/MyDrive/NLP AL Project (Data)/michael_data_clean_80k.json', 'r') as fh: 
  data = json.load(fh)

In [ ]:
#100 data 
from IPython.core.display import update_display
from google.colab import files
uploaded = files.upload()




Saving new_json_data.json to new_json_data.json


In [ ]:
import pandas as pd
import io

data = pd.read_json(io.StringIO(uploaded.get('new_json_data.json').decode('utf-8')))

In [ ]:
print(data)

            id                                            authors  \
0    1302.1617                  Philip Bull and Marc Kamionkowski   
1    1405.1350  Ofer Lahav (UCL) and Michela Massimi (Edinburg...   
2   1510.06248  J. Licandro (1,2), T. M\"uller (3), C. Alvarez...   
3    1412.7651      Ond\v{r}ej Kop\'a\v{c}ek and Vladim\'ir Karas   
4   1602.03242                                          Fan Zhang   
..         ...                                                ...   
61  2002.07260  Annalisa Calamida (1), Alice Zocchi (2), Giuse...   
62  1901.08471  Robert A. Wittenmyer, Jake T. Clark, Jinglin Z...   
63   0911.5648  J. Kainulainen (1), H. Beuther (1), T. Henning...   
64  1711.08176  Jiguang Lu, Enping Zhou, Xiaoyu Lai and Renxin Xu   
65   1305.6655  AMI Consortium: Timothy W. Shimwell, John M. C...   

                                       authors_parsed  \
0          [[Bull, Philip, ], [Kamionkowski, Marc, ]]   
1   [[Lahav, Ofer, , UCL], [Massimi, Michela, , Ed...   


In [ ]:
abstracts, categories, titles = data['abstract'], data['categories'], data['title']

##  Part 2: Text Normalization


*   Removes punctuations
*   Removes redundant whitespaces
*   Removes stopwords



In [ ]:
import re

In [ ]:
print(len(abstracts[0]))
#length of first abstract = 977. 

977


In [ ]:
#remove special characters
def removeSpecialChars(text, remove_digits=False):
  pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
  text = re.sub(pattern, '', text)
  return text

regexList = []
for index in range(0, len(abstracts)):
  regexList.append(removeSpecialChars(abstracts[index]))

print(len(regexList[0]))
#length of first abstract after removing special characters = 945; 3.21% decrease from initial value. 

945


In [ ]:
#remove redundant whitespaces
def removeExtraWhitespaces(text):
  text = re.sub(r'\s+'," ", text)
  return text.strip()

whitespaceList = []
for index in range(0, len(regexList)):
  whitespaceList.append(removeExtraWhitespaces(regexList[index]))

print(whitespaceList[0])
print(len(whitespaceList[0]))
#length of first abstract after removing redundant whitespaces = 940; 3.75$ decrease from initial value. 

Inflationary theory predicts that the observable Universe should be very close to flat with a spatialcurvature parameter OmegaK 104 The WMAP satellite currently constrains OmegaK 001 and the Planck satellite will be sensitive to values near 103 Suppose that Planck were to find OmegaK to be nonzero at this level Would this necessarily be a serious problem for inflation We argue that an apparent departure from flatness could be due either to a local wavelength comparable to the observable horizon inhomogeneity or a truly superhorizon departure from flatness If there is a local inhomogeneity then secondary CMB anisotropies distort the CMB frequency spectrum at a level potentially detectable by a nextgeneration experiment We discuss how these spectral distortions would complement constraints on the GrishchukZeldovich effect from the CMB power spectrum at large angular scales in discovering the source of the departure from flatness
940


In [ ]:
#use nltk's stopwords list. 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#remove stopwords
from nltk.tokenize.toktok import ToktokTokenizer

tokenizer = ToktokTokenizer()
stopWordList = set(nltk.corpus.stopwords.words('english'))

newWords = ['astrophysics', 'astronomy', 'analysis', 'comment', 'values']
stopWordList = stopWordList.union(newWords)

def removeStopwords(text, is_lower_case=False, stopwords=stopWordList):
  tokens= tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
    filteredTokens = [token for token in tokens if token not in stopwords]
  else: 
    filteredTokens = [token for token in tokens if token.lower() not in stopwords]
  filteredText = ' '.join(filteredTokens)
  return filteredText

noStopwordsList = []
for index in range(0, len(whitespaceList)):
  process = removeStopwords(whitespaceList[index])
  noStopwordsList.append(process)

print(noStopwordsList[0])
print(len(noStopwordsList[0]))
#length of first abstract after removing stopwords = 710; 27% decrease from the initial value. 

Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness
710


## Part 3: NLP Pipeline Implementation (spaCy)

In [ ]:
#implement spaCy pipeline
nlpAbstracts = []
for i in range(0, len(noStopwordsList)): 
  nlpAbstracts.append(nlpLg(noStopwordsList[i]))
  i += 1

print(nlpAbstracts)

[Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness, cases history may shed light current established enigmatic concordance model Cosmology model understood adding new entities Dark Matter Dark Energy modifying underlying theory example prediction discovery planet Neptune regarded analogous finding dark component explaining anomalous peri

## Part 4: Keyword Extractions


*   Term-Frequency (Naive Approach)
*   TF-IDF Extraction Model
*   Corpus & data frame initialization 


In [ ]:
#based on our interest, we will only extract nouns, numerics, proper nouns and symbols. 
def getKeywords(doc):
  result = []
  posTag = ['NOUN', 'NUM', 'PROPN', 'SYM']

  for token in doc: 
    #from tokens, we will re-remove punctuations for double-filtering. 
    if (token.text in nlpLg.Defaults.stop_words or token.text in punctuation):
      continue

    if (token.pos_ in posTag):
      result.append(token.text)

  return result

In [ ]:
####### tester case #########
#here, we will only practice our getKeywords function for the first abstract after spaCy NLP Pipeline. 
output = getKeywords(nlpAbstracts[0])
print(output)

#remove duplicates by using set instead of array. 
output = set(getKeywords(nlpAbstracts[0]))
print(output)
print(len(output))

#total numbers of abstracts[0] was 977 and keywords here are saturated to 32. 3.58% of total # of words. 

['theory', 'Universe', 'parameter', '104', 'WMAP', 'satellite', '001', 'Planck', 'satellite', '103', 'Suppose', 'Planck', 'nonzero', 'level', 'problem', 'inflation', 'departure', 'flatness', 'wavelength', 'horizon', 'inhomogeneity', 'departure', 'flatness', 'inhomogeneity', 'CMB', 'anisotropies', 'CMB', 'frequency', 'spectrum', 'level', 'nextgeneration', 'experiment', 'distortions', 'constraints', 'GrishchukZeldovich', 'effect', 'CMB', 'power', 'spectrum', 'scales', 'source', 'departure', 'flatness']
{'Universe', 'level', 'flatness', 'parameter', 'experiment', 'Planck', 'inhomogeneity', 'problem', 'satellite', 'source', 'spectrum', 'CMB', 'distortions', 'GrishchukZeldovich', '001', 'WMAP', 'inflation', '104', 'departure', 'effect', '103', 'scales', 'anisotropies', 'theory', 'constraints', 'nextgeneration', 'nonzero', 'horizon', 'frequency', 'power', 'Suppose', 'wavelength'}
32


In [ ]:
######## tester case ##########
#extract the 32 keywords to top 5 "most common" keywords. 
topTen = [keyword for keyword in Counter(output).most_common(5)]
print(topTen)

[('Universe', 1), ('level', 1), ('flatness', 1), ('parameter', 1), ('experiment', 1)]


In [ ]:
#keyword bank
def keywordExtractor(text):
  output = set(getKeywords(text))
  return output

keywordBank = []
for index in range(0, len(nlpAbstracts)):
  process = keywordExtractor(nlpAbstracts[index])
  keywordBank.append(process)

print(keywordBank[0])
print(len(keywordBank[0]))

{'Universe', 'level', 'flatness', 'parameter', 'experiment', 'Planck', 'inhomogeneity', 'problem', 'satellite', 'source', 'spectrum', 'CMB', 'distortions', 'GrishchukZeldovich', '001', 'WMAP', 'inflation', '104', 'departure', 'effect', '103', 'scales', 'anisotropies', 'theory', 'constraints', 'nextgeneration', 'nonzero', 'horizon', 'frequency', 'power', 'Suppose', 'wavelength'}
32


So far, we've measured the keywords by term-frequency from each abstract. However, this does not take consideration of "contextual knowledge" that are considered to be saturated with the "real keywords." Hence, we will use two different methods: Term-Frequency Inverse-Term-Frequency (TF-IDF) and text embedding. 

In [ ]:
#make corpus array that consists of string abstracts (docs) after spaCy's NLP Pipeline. 
corpus = []

for index in range(0, len(nlpAbstracts)):
  text = ' '.join(str(v) for v in nlpAbstracts[index])
  corpus.append(text)

print(corpus[0])

Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness


In [ ]:
#create a pandas dataframe for better presentations. Each index is assigned by title, category, keywords and abstract. 
#keywords will be modified to best "fit" model. 
import pandas as pd

keywordBankResult = pd.DataFrame()
keywordBankResult['Title'] = titles
keywordBankResult['Category'] = categories
keywordBankResult['Keywords'] = keywordBank
keywordBankResult['Abstract'] = corpus

print(keywordBankResult[:5])

                                               Title  \
0              What if Planck's Universe isn't flat?   
1  Dark Energy, Paradigm Shifts, and the Role of ...   
2      CanariCam/GTC observations of (99942) Apophis   
3  Regular and chaotic motion in general relativi...   
4  Intrinsic electromagnetic variability in celes...   

                      Category  \
0            astro-ph.CO gr-qc   
1  physics.hist-ph astro-ph.CO   
2                  astro-ph.EP   
3                  astro-ph.HE   
4                  astro-ph.HE   

                                            Keywords  \
0  {Universe, level, flatness, parameter, experim...   
1  {cases, components, Cosmology, similarities, g...   
2  {Palma, PACS, 100, pm, hazard, Apophis, los, f...   
3  {study, largescale, exponents, vicinity, condi...   
4  {orders, fit, observations, seed, paper, expul...   

                                            Abstract  
0  Inflationary theory predicts observable Univer...  
1  cases hi

First, we will practice TF-IDF keyword extraction model.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import pandas as pd
import re

In [ ]:
def BasicTFIDF(abstract):
  sentences = list()
  for l in re.split(r"\.\s|\?\s|\!\s|\n", abstract):
    if l: 
      sentences.append(l)

  cVec = CountVectorizer(stop_words='english', min_df=3, max_df=0.5, ngram_range=(1,2))
  sf = cVec.fit_transform(sentences)
  
  transformer = TfidfTransformer()
  transformedWeights = transformer.fit_transform(sf)
  weights = np.asarray(transformedWeights.mean(axis=0)).ravel().tolist()
  weights_df = pd.DataFrame({'term': cVec.get_feature_names(), 'weight': weights})

  visual = weights_df.sort_values(by='weight', ascending=False).head(10)
  return visual

BasicTFIDF(abstracts[0])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,weight
3,omega_k,0.157895
0,departure,0.091161
1,departure flatness,0.091161
2,flatness,0.091161


As observed, considering Inverse Term-Frequency (IDF portion of TF-IDF) shows different result than the model above. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

CV = CountVectorizer(max_df=0.8, stop_words=stopWordList, max_features=10000, ngram_range=(1,3))
X = CV.fit_transform(corpus)

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline

So far, we've created a keyword extractor that outputs most commonly used "keyword" from the given abstract. However, we will run a TF-IDF testing to ensure the validity of those keywords. 

Now, we know that "omega_k" is the most "relevant" in the context of abstracts[0]. We reckon the top 10 most frequent words were all occurred once, hence this TF-IDF testing gives us a better context of which words are the most "important" for our classifiers. 

Moving forward, we will create a set of "keywords" for each astro-ph category to classify which research paper belongs to which category.

https://iyzico.engineering/how-to-calculate-tf-idf-term-frequency-inverse-document-frequency-from-the-beatles-biography-in-c4c3cd968296

https://betterprogramming.pub/extract-keywords-using-spacy-in-python-4a8415478fbf

https://arxiv.org/archive/astro-ph

## Part 5: Keyword Bank (with Embedding)

By using BERT, we will support computer to understand ambiguous language context by surrounding text to create "context." We will pre-train our corpus. 

Current Plan: 

1.   Embeddings: position embedding, segment embedding, and token embeddings. 
2.   Pre-Training: masked language modelling (bi-directional), next sentence prediction
3.   Keyword Bank Implementation 



In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 235 kB 11.9 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 
     |████████████████████████████████| 4.7 MB 48.5 MB/s 
     |████████████████████████████████| 1.2 MB 49.7 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 596 kB 60.3 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b0aa06a00044229d270b6a8c2fe8dcbb96d87aa8edb3c0690be066980990053b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      

In [ ]:
from keybert import KeyBERT

#1 & 2-grams are considered here (preserving key phrases)
keywordModel = KeyBERT()
keywordsNGram = keywordModel.extract_keywords(corpus[0], keyphrase_ngram_range=(1, 2))
print(keywordsNGram)
print(len(keywordsNGram))

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

[('inflationary theory', 0.5525), ('observable universe', 0.4645), ('inflationary', 0.4617), ('observable horizon', 0.4474), ('planck satellite', 0.4385)]
5


In [ ]:
#diversified keywords & phrases. 
keywordsDiversified = keywordModel.extract_keywords(corpus[0], keyphrase_ngram_range=(1, 2), use_maxsum=True, nr_candidates=20, top_n=5)
print(keywordsDiversified)

[('cmb frequency', 0.3265), ('flatness local', 0.3368), ('inflation argue', 0.3604), ('spectral distortions', 0.4162), ('observable horizon', 0.4474)]


In [ ]:
#for potential foreign linguistics. 
keywordModelForeign = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')
keywordsForeign = keywordModel.extract_keywords(corpus[0], keyphrase_ngram_range=(1, 2))
print(keywordsForeign)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[('inflationary theory', 0.5525), ('observable universe', 0.4645), ('inflationary', 0.4617), ('observable horizon', 0.4474), ('planck satellite', 0.4385)]


In [ ]:
#make a loop & modify keywordBank. 
keywordModel = KeyBERT()
def keywordExtractionforNgram(text):
  keywordsNGram1 = keywordModel.extract_keywords(text, keyphrase_ngram_range=(1, 2))

  return keywordsNGram1

def keywordExtractionforDiversification(text):
  keywordsDiversified2 = keywordModel.extract_keywords(text, keyphrase_ngram_range=(1, 2), use_maxsum=True, nr_candidates=20, top_n=5)
  return keywordsDiversified2

nGramKeywordList = []
diversifiedKeywordList = []
for index in range(0, len(corpus)):
  nGramKeywordList.append(keywordExtractionforNgram(corpus[index]))
  diversifiedKeywordList.append(keywordExtractionforDiversification(corpus[index]))

print(diversifiedKeywordList[:3])

[[('cmb frequency', 0.3265), ('flatness local', 0.3368), ('inflation argue', 0.3604), ('spectral distortions', 0.4162), ('observable horizon', 0.4474)], [('cases history', 0.3668), ('concordance model', 0.3695), ('similarities differences', 0.3765), ('underlying theory', 0.3981), ('dark energy', 0.4424)], [('thermal inertia', 0.3107), ('observatory pacs', 0.327), ('pv thermal', 0.3373), ('geometric albedo', 0.3869), ('hazardous asteroid', 0.5996)]]


In [ ]:
print(len(diversifiedKeywordList))

print(diversifiedKeywordList[0])
print(diversifiedKeywordList[1])

66
[('cmb frequency', 0.3265), ('flatness local', 0.3368), ('inflation argue', 0.3604), ('spectral distortions', 0.4162), ('observable horizon', 0.4474)]
[('cases history', 0.3668), ('concordance model', 0.3695), ('similarities differences', 0.3765), ('underlying theory', 0.3981), ('dark energy', 0.4424)]


In [ ]:
#modified keyword bank with KeyBERT model
newKeywordBank = pd.DataFrame()

newKeywordBank['Titles'] = titles
newKeywordBank['Categories'] = categories
newKeywordBank['BERT nGram Keywords'] = nGramKeywordList
newKeywordBank['BERT Diversified Keywords'] = diversifiedKeywordList
newKeywordBank['Abstracts'] = corpus

print(newKeywordBank[:3])

                                              Titles  \
0              What if Planck's Universe isn't flat?   
1  Dark Energy, Paradigm Shifts, and the Role of ...   
2      CanariCam/GTC observations of (99942) Apophis   

                    Categories  \
0            astro-ph.CO gr-qc   
1  physics.hist-ph astro-ph.CO   
2                  astro-ph.EP   

                                 BERT nGram Keywords  \
0  [(inflationary theory, 0.5525), (observable un...   
1  [(cosmology, 0.5211), (dark matter, 0.4904), (...   
2  [(asteroid pha, 0.6076), (hazardous asteroid, ...   

                           BERT Diversified Keywords  \
0  [(cmb frequency, 0.3265), (flatness local, 0.3...   
1  [(cases history, 0.3668), (concordance model, ...   
2  [(thermal inertia, 0.3107), (observatory pacs,...   

                                           Abstracts  
0  Inflationary theory predicts observable Univer...  
1  cases history may shed light current establish...  
2  potentially hazardou

Although KeyBERT is an efficient embedding model itself, this is a very "surface level technique" that is not complex enough for our models to leverage. 

Hence, we will try different types of technologies like YAKE, RAKE, Word2Vec, GloVe, and GPT-3 to compare its effectiveness and efficiency. 

Article (SH Park): https://blog.pingpong.us/transformer-review/#:~:text=GPT%EC%99%80%20BERT%EC%9D%98%20%EC%84%B1%EC%A7%88%EA%B3%BC%20%EB%B9%84%EA%B5%90,-Vaswani%20et%20al&text=%EB%91%90%20%EA%B5%AC%EC%A1%B0%EC%9D%98%20%EA%B0%80%EC%9E%A5%20%ED%81%B0,%EC%9E%88%EB%8B%A4%EB%8A%94%20%EA%B0%95%EC%A0%90%EC%9D%84%20%EA%B0%80%EC%A7%91%EB%8B%88%EB%8B%A4.

In [ ]:
#to json output
dataWithKeywords = {}

for index in range(0, len(newKeywordBank)):
  dataWithKeywords[titles[index]] = {'Category': categories[index], 'Abstract': corpus[index], 'BERT nGram Keywords': nGramKeywordList[index], 'BERT Diversified Keywords': diversifiedKeywordList[index]}

print(dataWithKeywords)

{"What if Planck's Universe isn't flat?": {'Category': 'astro-ph.CO gr-qc', 'Abstract': 'Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness', 'BERT nGram Keywords': [('inflationary theory', 0.5525), ('observable universe', 0.4645), ('inflationary', 0.4617), ('observable horizon', 0.4474), ('planck satellite', 0.4385)], 'BERT Diversified K

In [ ]:
from pandas.core.frame import DataFrame
df = DataFrame(dataWithKeywords)
#df.to_json(r'C:\Users\Michael\Downloads\100Keywords.json')


In [ ]:
print(df)

                                       What if Planck's Universe isn't flat?  \
Category                                                   astro-ph.CO gr-qc   
Abstract                   Inflationary theory predicts observable Univer...   
BERT nGram Keywords        [(inflationary theory, 0.5525), (observable un...   
BERT Diversified Keywords  [(cmb frequency, 0.3265), (flatness local, 0.3...   

                          Dark Energy, Paradigm Shifts, and the Role of Evidence  \
Category                                         physics.hist-ph astro-ph.CO       
Abstract                   cases history may shed light current establish...       
BERT nGram Keywords        [(cosmology, 0.5211), (dark matter, 0.4904), (...       
BERT Diversified Keywords  [(cases history, 0.3668), (concordance model, ...       

                               CanariCam/GTC observations of (99942) Apophis  \
Category                                                         astro-ph.EP   
Abstract          

In [ ]:
with open('df.json', 'w') as f:
  f.write(df.to_json())

In [ ]:
!cat df.json

{"What if Planck's Universe isn't flat?":{"Category":"astro-ph.CO gr-qc","Abstract":"Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness","BERT nGram Keywords":[["inflationary theory",0.5525],["observable universe",0.4645],["inflationary",0.4617],["observable horizon",0.4474],["planck satellite",0.4385]],"BERT Diversified Keywords":[["cmb 

In [ ]:
from google.colab import files
files.download('df.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 19.6 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73986 sha256=27689a42301c6ffe0e9aa6e76884d1f0b9db7a0eb953e351e08ec29d2cfbccef
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
#YAKE
import yake

extraction = yake.KeywordExtractor()
language = 'en'
maxNgramSize = 3
deduplicationThreshold = 0.9
numKeywords = 3
keywordExtractor = yake.KeywordExtractor(lan=language, n=maxNgramSize, dedupLim=deduplicationThreshold, top=numKeywords,features=None)

yakeList = []
for index in range(0, len(corpus)):
  keywords = keywordExtractor.extract_keywords(corpus[index])
  yakeList.append(keywords)


print(yakeList[0])
print(len(yakeList[0]))

[('Suppose Planck find', 0.0007755501592003004), ('Universe close flat', 0.001631106699069244), ('Inflationary theory predicts', 0.002127673336471348)]
3


Since capturing context from the corpus is our goal, we will use ELMo and BERT to generate text embeddings that holds account of context of a word (position of sentence), although the process is complicated and expensive. We will take account of word order (ELMo - LSTM, BERT - transformer). 

ELMo --> contrast character based model by character convolutions that handles vocabulary. Context sensitive embedding, yet words are learnt representations. 

BERT --> learns embeddings for subwords, balance between character & word based representations. 

In [ ]:
#astro-ph categorization

#astrophysics of galaxies
astro_ph_GA = {'galaxy', 'milky way', 'star cluster', 'planetary nebulae', 'interstellar medium', 'atomic cloud', 'molecular cloud', 'stellar populations', 'galactic structure', 'galactic nuclei', 'bulge', 'halo', 'black hole', 'quasar'}
#cosmology and nongalactic astrophysics
astro_ph_CO = {'cosmology', 'microwave background', 'cosmological parameter', 'primordial abundance', 'extragalactic distance', 'intergalactic medium', 'baryogenesis', 'leptogenesis', 'gravitational radiation'}
#earth and planetary astrophysics
astro_ph_EP = {'interplanetary medium', 'planetary physics', 'planetary astrobiology', 'comet', 'asteroid', 'meteorite'}
#high energy astrophysical phenomena
astro_ph_HE = {'energy', 'cosmic ray', 'acceleration', 'propagation', 'gamma ray', 'xray', 'supernovae', 'jet', 'microquarsar', 'neutron star', 'pulsar'}
#instrumentation and methods of astrophysics
astro_ph_IM = {'telescope', 'laboratory', 'data analysis', 'statistical method', 'software', 'database'}
#solar and stellar astrophysics
astro_ph_SR = {'white dwarf', 'brown dwarf', 'cataclysmic variable', 'stellar evolution', 'helioseismology', 'gravitational radiation'}

In [ ]:
#categorization
 #https://towardsdatascience.com/using-keyword-extraction-for-unsupervised-text-classification-in-nlp-10433a1c0cf9
 #https://www.toptal.com/machine-learning/nlp-tutorial-text-classification
 #https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db
 #https://towardsdatascience.com/keybert-keyword-extraction-using-bert-a6dc3dd38caf
 

To Do


*   Text classification by categories. 
*   Keywordbank


*   GloVe
*   List item





## Part 4: Data preparation

--------------------Below is Current Progress---------------------

In [ ]:
#Categorization
import pandas as pd
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
d = {'Title': titles, 'Category': categories, 'Abstract': nlpAbstracts}
df = pd.DataFrame(d, columns=['Title', 'Category', 'Abstract'])
print(df)

                                                                                                                      Title  \
0                                                                                     What if Planck's Universe isn't flat?   
1                                                                    Dark Energy, Paradigm Shifts, and the Role of Evidence   
2                                                                             CanariCam/GTC observations of (99942) Apophis   
3                     Regular and chaotic motion in general relativity: The case of an\n  inclined black hole magnetosphere   
4                     Intrinsic electromagnetic variability in celestial objects containing\n  rapidly spinning black holes   
..                                                                                                                      ...   
61                             The not so simple stellar system Omega Cen. II. Evidence in support of a\n  merg

In [ ]:
#tokenized

textCorpus = df['Abstract'].values
taggedCorpus = [TaggedDocument(d, [i])for i, d in enumerate(textCorpus)]

print(taggedCorpus[0])

TaggedDocument(Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive values near 103 Suppose Planck find OmegaK nonzero level Would necessarily serious problem inflation argue apparent departure flatness could due either local wavelength comparable observable horizon inhomogeneity truly superhorizon departure flatness local inhomogeneity secondary CMB anisotropies distort CMB frequency spectrum level potentially detectable nextgeneration experiment discuss spectral distortions would complement constraints GrishchukZeldovich effect CMB power spectrum large angular scales discovering source departure flatness, [0])


In [ ]:
#wire categories to set of descriptions



In [ ]:
#keyword bank

# Agenda: 


1.   n-gram
2.   List item

https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

# Preprocessing

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split

# TF-IDF Vectors as Features

In [ ]:
#load data. 

#we will sentencize each abstract and provide them with a label (per abstract)
labels = []
texts = []

for index in range(0, len(nlpAbstracts)):
  labels.append(categories[0])

  sentList = []
  for sent in nlpAbstracts[index]:
    sentList.append(sent.text)
  
  texts.append(' '.join(sentList))

print(labels)
print(len(texts))

#creates dataframe using texts (tokenized) and labels (categories)
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

print(trainDF)

['astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO gr-qc', 'astro-ph.CO

In [ ]:
#now, split the dataset to train & test the classifier. 

trainX, validX, trainY, validY = model_selection.train_test_split(trainDF['text'], trainDF['label'])

encoder = preprocessing.LabelEncoder()
trainY = encoder.fit_transform(trainY)
validY = encoder.fit_transform(validY)


# Count Vectors as Features

In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

xtrain_count =  count_vect.transform(trainX)
xvalid_count =  count_vect.transform(validX)

print(xtrain_count, xvalid_count)

  (0, 115)	1
  (0, 146)	1
  (0, 270)	1
  (0, 271)	1
  (0, 345)	2
  (0, 348)	1
  (0, 361)	1
  (0, 378)	2
  (0, 429)	1
  (0, 454)	1
  (0, 468)	1
  (0, 496)	1
  (0, 618)	3
  (0, 623)	3
  (0, 635)	1
  (0, 640)	1
  (0, 661)	1
  (0, 871)	1
  (0, 885)	1
  (0, 1091)	1
  (0, 1191)	1
  (0, 1275)	1
  (0, 1282)	1
  (0, 1283)	1
  (0, 1284)	1
  :	:
  (48, 1646)	1
  (48, 1777)	1
  (48, 1793)	1
  (48, 1842)	1
  (48, 1900)	1
  (48, 1951)	1
  (48, 1991)	1
  (48, 2034)	1
  (48, 2051)	1
  (48, 2095)	1
  (48, 2134)	1
  (48, 2135)	1
  (48, 2262)	2
  (48, 2410)	1
  (48, 2436)	1
  (48, 2441)	1
  (48, 2442)	1
  (48, 2528)	1
  (48, 2548)	3
  (48, 2572)	1
  (48, 2627)	1
  (48, 2856)	1
  (48, 2875)	1
  (48, 2897)	1
  (48, 2903)	2   (0, 201)	2
  (0, 202)	1
  (0, 203)	1
  (0, 262)	1
  (0, 265)	1
  (0, 268)	1
  (0, 270)	1
  (0, 319)	1
  (0, 364)	1
  (0, 366)	1
  (0, 403)	2
  (0, 406)	1
  (0, 408)	2
  (0, 424)	1
  (0, 597)	1
  (0, 598)	1
  (0, 620)	1
  (0, 628)	1
  (0, 656)	1
  (0, 672)	1
  (0, 704)	1
  (0, 746)	1
  

# TF-IDF Vectors as Features



*   Word level: matrix representing TF-IDF scores of every item in different abstracts. 
*   N-Gram level: N-Grams = combination of N-terms. Represents TF-IDF scores of N-Grams
*   Character level: matrix representing TF-IDF scores of character level N-Grams in our corpus. 





In [ ]:
#word level
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf = tfidf_vect.transform(trainX)
xvalid_tfidf = tfidf_vect.transform(validX)

In [ ]:
#N-Gram level
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(trainX)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(validX)

In [ ]:
#characters level
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(trainX)
xvalid_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(validX)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  "The parameter 'token_pattern' will not be used"


# Word Embedding


In [ ]:
import string
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
print(trainDF)

                                                                                                                                                                                                       text  \
0   Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive values near 103 Suppose ...   
1   comment cases history Astronomy may shed light current established enigmatic concordance model Cosmology model understood adding new entities Dark Matter Dark Energy modifying underlying theory ex...   
2   potentially hazardous asteroid PHA 99942 Apophis one remarkable nearEarth asteroids NEA terms impact hazard good determination surface thermal inertia important order evaluate Yarkovsky effect orb...   
3   Dynamics charged particles vicinity rotating black hole embedded external largescale magnetic field numerically investigated particular consider nonaxisymmetric model a

In [ ]:
import string
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [ ]:
print(trainDF)

                                                                                                                                                                                                       text  \
0   Inflationary theory predicts observable Universe close flat spatialcurvature parameter OmegaK 104 WMAP satellite currently constrains OmegaK 001 Planck satellite sensitive values near 103 Suppose ...   
1   comment cases history Astronomy may shed light current established enigmatic concordance model Cosmology model understood adding new entities Dark Matter Dark Energy modifying underlying theory ex...   
2   potentially hazardous asteroid PHA 99942 Apophis one remarkable nearEarth asteroids NEA terms impact hazard good determination surface thermal inertia important order evaluate Yarkovsky effect orb...   
3   Dynamics charged particles vicinity rotating black hole embedded external largescale magnetic field numerically investigated particular consider nonaxisymmetric model a

In [ ]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(len(topic_summaries))
print(topic_summaries)

20
['stars mainsequence enrichment li blue omega cen red one engulfment', 'flames pulsation photopic m2 temperatures luminance small spectral power different', 'dispersion elements rprocess m92 present gc abundances dex metalpoor m15', 'universe backreaction model additional cosmic energy ryskin ryskins gravity source', 'evolution cosmography universe approximant since one pade several inspired work', 'li enrichment engulfment hj stars using cii events branch strengths', 'planets massperiod relation orbits orbital statistical ensemble circularisation closein circular', 'ring gas abundances events stellar disk sep aq formation data', 'model data mass optical des disk emission distribution shock survey', 'rm galaxies line luvoir surveyor b0 mission absorption magnetic chisl', 'box shearing mhd simulation radial velocity cartesian mass magnetic flux', 'column density pressure clouds cii cluster pdfs molecular gas using', 'nm thermal uv data space polymer apophis inertia light albedo', 'bo

#Text Based Features

# Topic Models as Features

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn import decomposition
#Latent Dirichlet Allocation (LDA)

ldaModel = decomposition.LatentDirichletAllocation(n_components=6, learning_method='online', max_iter=6)
xTopics = ldaModel.fit_transform(xtrain_count)
topicWord = ldaModel.components_
vocab = CountVectorizer.get_feature_names

print(xTopics)

[[2.45685161e-03 9.87701779e-01 2.46116113e-03 2.46075334e-03
  2.46104632e-03 2.45840828e-03]
 [1.30552779e-03 9.93470484e-01 1.30681646e-03 1.30573823e-03
  1.30511289e-03 1.30632036e-03]
 [1.50714947e-03 1.50733867e-03 1.50812791e-03 1.50462285e-03
  1.50531362e-03 9.92467447e-01]
 [2.14981312e-03 2.14356126e-03 2.14379443e-03 2.14373799e-03
  2.14227535e-03 9.89276818e-01]
 [1.56604940e-03 1.56340480e-03 9.92183192e-01 1.56125910e-03
  1.56343863e-03 1.56265653e-03]
 [2.50542393e-03 9.87501707e-01 2.50503749e-03 2.49636727e-03
  2.49443098e-03 2.49703305e-03]
 [9.94677291e-01 1.06452396e-03 1.06421745e-03 1.06487181e-03
  1.06411428e-03 1.06498120e-03]
 [1.74649106e-03 9.91279608e-01 1.74602082e-03 1.74097721e-03
  1.74230218e-03 1.74460098e-03]
 [9.40226861e-04 9.40405188e-04 9.95301267e-01 9.39389417e-04
  9.38854205e-04 9.39857430e-04]
 [1.50910455e-03 1.51010635e-03 1.50817040e-03 1.50630158e-03
  1.50773900e-03 9.92458578e-01]
 [1.13917875e-03 1.13881938e-03 1.13824810e-03 1.1

Keyword Extractions

In [ ]:
import json

with open('/content/drive/MyDrive/NLP AL Project (Data)/michael_data_clean_80k.json', 'r') as fh: 
  data = json.load(fh)

In [ ]:
data[:10]

[{'abstract': '  We present a theoretical framework for plasma turbulence in astrophysical\nplasmas (solar wind, interstellar medium, galaxy clusters, accretion disks).\nThe key assumptions are that the turbulence is anisotropic with respect to the\nmean magnetic field and frequencies are low compared to the ion cyclotron\nfrequency. The energy injected at the outer scale scale has to be converted\ninto heat, which ultimately cannot be done without collisions. A KINETIC\nCASCADE develops that brings the energy to collisional scales both in space and\nvelocity. Its nature depends on the physics of plasma fluctuations. In each of\nthe physically distinct scale ranges, the kinetic problem is systematically\nreduced to a more tractable set of equations. In the "inertial range" above the\nion gyroscale, the kinetic cascade splits into a cascade of Alfvenic\nfluctuations, which are governed by the RMHD equations at both the collisional\nand collisionless scales, and a passive cascade of comp

# Weekly To-Do List



*   Monday: text normalization, keyword bank
*   Tuesday: text vectorization
*   Wednesday: neural network
*   Thursday: neural network
*   Friday: neural network


https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/nlp/dl-neural-network-from-scratch.html


